In [ ]:
## This file is part of Jax Geometry
#
# Copyright (C) 2021, Stefan Sommer (sommer@di.ku.dk)
# https://bitbucket.org/stefansommer/jaxgeometry
#
# Jax Geometry is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# Jax Geometry is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Jax Geometry. If not, see <http://www.gnu.org/licenses/>.
#

# Stochastic Lie group dynamics

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# SO(3)
from src.groups.SON import *
G = SON(3)
print(G)

# SO(3) acts on S^2
from src.manifolds.S2 import *
M = S2()
print(M)

from src.plotting import *

In [ ]:
# setup for testing different versions of stochastic dynamics
q = jnp.array([1e-3,0.,0.])
g = G.psi(q)
v = jnp.array([0.,1.,1.])

x = M.coords(jnp.array([0.,0.]))

from src.group import invariant_metric
invariant_metric.initialize(G)
p = G.sharppsi(q,v)
mu = G.sharpV(v)
print(p)
print(mu)

from src.group import energy
energy.initialize(G)

In [ ]:
# Brownian motion
from src.stochastics import Brownian_inv
Brownian_inv.initialize(G)

_dts = dts(n_steps=1000)
(ts,gs,_) = G.Brownian_inv(g,_dts,dWs(G.dim,_dts))

# plot
newfig()
G.plot_path(gs,linewidth=0.1,alpha=0.1)
plt.show()
#plt.savefig('stocso3.pdf')

# on S2
newfig()
M.plot()
M.plot_path(M.acts(gs,M.F(x)))
plt.show()
#plt.savefig('stocso32.pdf')

In [ ]:
# Brownian processes
from src.stochastics import Brownian_process
Brownian_process.initialize(G)

_dts = dts(n_steps=1000)
(ts,gs,_) = G.Brownian_process(g,_dts,dWs(G.dim,_dts))

# plot
newfig()
G.plot_path(gs,color_intensity=1,alpha=0.1)
plt.show()

In [ ]:
# Euler-Poincare dynamics
from src.group import EulerPoincare
EulerPoincare.initialize(G)# Euler-Poincare dynamics

In [ ]:
# Stochastic coadjoint motion
from src.stochastics import stochastic_coadjoint
stochastic_coadjoint.initialize(G)

_dts = dts(n_steps=1000)
(ts,mus,_) = G.stochastic_coadjoint(mu,_dts,dWs(G.dim,_dts))
(ts,gs) = G.stochastic_coadjointrec(g,mus,_dts)

# plot
newfig()
G.plot_path(gs,color_intensity=1,alpha=0.1)
plt.show()
#plt.savefig('coadgeo.pdf')

In [ ]:
# Delyon/Hu guided process
from src.stochastics.guided_process import *

# guide function
phi = lambda g,v,sigma: jnp.tensordot(G.inv(sigma),G.LAtoV(G.log(G.invtrns(G.inv(g[0]),v))),(1,0))
A = lambda g,v,w,sigma: G.gG(g[0],v,w,sigma)
logdetA = lambda x,sigma: -(jnp.linalg.slogdet(s*sigma)[1])**2

(Brownian_inv_guided,sde_Brownian_coords_inv,_,_,_) = get_guided(
    G,G.sde_Brownian_inv,None,phi,
    lambda g,sigma: sigma,A,logdetA,integration='stratonovich')

_dts = dts(n_steps=1000)
(ts,gs,_,log_likelihood,log_varphi) = Brownian_inv_guided((g,None),G.psi(v),_dts,dWs(G.dim,_dts),
                                                          jnp.sqrt(.2)*jnp.eye(G.dim))
print("log likelihood: ", log_likelihood[-1], ", log varphi: ", log_varphi[-1])

newfig()
w = G.psi(v)
G.plot_path(gs,linewidth=0.1,alpha=0.1)
G.plotg(w,color='k')
plt.show()

# on S2
newfig()
M.plot()
M.plot_path(M.acts(gs,M.F(x)))
M.plotx(M.act(w,M.F(x)),color='k')
plt.show()